In [1]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)

from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()

#hugging face token 입력


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to /home/20223149/.cache/huggingface/token
Login successful


In [58]:
huggingface_dataset_name = "beomi/KoAlpaca-v1.1a"
#data set load
dataset = load_dataset(huggingface_dataset_name)

In [2]:
from datasets import DatasetDict

# 기존 DatasetDict 로드 또는 정의
dataset_dict = dataset

# 'train' 데이터셋을 'train'과 'temp'로 분할 (예: 70% 'train', 30% 'temp')
train_testsplit = dataset_dict['train'].train_test_split(test_size=0.3)

# 'temp'를 'validation'과 'test'로 분할 (예: 각각 'temp'의 50%, 총 데이터셋의 15%)
val_testsplit = train_testsplit['test'].train_test_split(test_size=0.5)

# 새로운 DatasetDict 생성
dataset = DatasetDict({
    'train': train_testsplit['train'],  # 70% 데이터
    'validation': val_testsplit['train'],  # 15% 데이터
    'test': val_testsplit['test']  # 나머지 15% 데이터
})

NameError: name 'dataset' is not defined

In [1]:
dataset['train'][42]

NameError: name 'dataset' is not defined

In [61]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 14808
    })
    validation: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 3173
    })
    test: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 3174
    })
})

In [62]:
len(dataset['train'])

14808

In [63]:
compute_dtype = getattr(torch, "float16")
#모델을 4bit 형식으로 로드(메모리 소비가 줄어듦)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [64]:
model_name='microsoft/phi-2'
device_map = {"": 0}
#모델을 양자화하여 다운(or load)
original_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [65]:
#hugging face의 transformers 라이브로리를 사용하여 자연어 처리 모델에 대한 토크나이저를 설정하는 함수
#AutoTokenizer.from_pretrained: 사전 훈련된 토크나이저를 불러옵니다.
#model_name: 사용할 사전 훈련된 모델의 이름 또는 경로를 지정합니다.
#trust_remote_code=True: 원격 코드를 신뢰하도록 지정합니다. 이것은 Hugging Face 모델 허브에서 토크나이저를 다운로드할 때 사용됩니다.
#padding_side="left": 패딩을 왼쪽에 추가합니다. 이는 토크나이저가 입력 시퀀스를 패딩할 때 어느 쪽에 패딩을 추가할지를 지정합니다.
#add_eos_token=True: End-Of-Sequence (EOS) 토큰을 추가합니다. 이는 문장의 끝을 나타내는 특수 토큰입니다.
#add_bos_token=True: Begin-Of-Sequence (BOS) 토큰을 추가합니다. 이는 문장의 시작을 나타내는 특수 토큰입니다.
#use_fast=False: 빠른 토크나이저를 사용하지 않도록 설정합니다. 이것은 더 빠른 토크나이저가 아닌 표준 토크나이저를 사용하도록 강제합니다.
tokenizer_ko = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b",trust_remote_code=True,padding_side="right", padding=True,add_eos_token=True,add_bos_token=True,use_fast=False)
#tokenizer.pad_token = tokenizer.eos_token: 이 부분은 패딩 토큰을 EOS (End-Of-Sequence) 토큰으로 설정합니다. 이렇게 하면 모델이 패딩을 식별하는 데 사용되는 토큰을 EOS 토큰으로 사용하게 됩니다.
tokenizer_ko.pad_token = tokenizer_ko.eos_token

#이 코드는 주어진 모델의 토크나이저를 설정하고, EOS 및 BOS 토큰을 추가하며, 패딩을 왼쪽에 추가하도록 구성합니다.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [66]:
#hugging face의 transformers 라이브로리를 사용하여 자연어 처리 모델에 대한 토크나이저를 설정하는 함수
#AutoTokenizer.from_pretrained: 사전 훈련된 토크나이저를 불러옵니다.
#model_name: 사용할 사전 훈련된 모델의 이름 또는 경로를 지정합니다.
#trust_remote_code=True: 원격 코드를 신뢰하도록 지정합니다. 이것은 Hugging Face 모델 허브에서 토크나이저를 다운로드할 때 사용됩니다.
#padding_side="left": 패딩을 왼쪽에 추가합니다. 이는 토크나이저가 입력 시퀀스를 패딩할 때 어느 쪽에 패딩을 추가할지를 지정합니다.
#add_eos_token=True: End-Of-Sequence (EOS) 토큰을 추가합니다. 이는 문장의 끝을 나타내는 특수 토큰입니다.
#add_bos_token=True: Begin-Of-Sequence (BOS) 토큰을 추가합니다. 이는 문장의 시작을 나타내는 특수 토큰입니다.
#use_fast=False: 빠른 토크나이저를 사용하지 않도록 설정합니다. 이것은 더 빠른 토크나이저가 아닌 표준 토크나이저를 사용하도록 강제합니다.
tokenizer_phi_2 = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="right", padding=True,add_eos_token=True,add_bos_token=True,use_fast=False)
#tokenizer.pad_token = tokenizer.eos_token: 이 부분은 패딩 토큰을 EOS (End-Of-Sequence) 토큰으로 설정합니다. 이렇게 하면 모델이 패딩을 식별하는 데 사용되는 토큰을 EOS 토큰으로 사용하게 됩니다.
tokenizer_phi_2.pad_token = tokenizer_phi_2.eos_token

#이 코드는 주어진 모델의 토크나이저를 설정하고, EOS 및 BOS 토큰을 추가하며, 패딩을 왼쪽에 추가하도록 구성합니다.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [67]:
def gen_ko(model_name, prompt, max_length):
    # 입력 텍스트를 토크나이즈하고 모델에 입력할 형식으로 변환
    input_ids = tokenizer_ko.encode(prompt, max_length=max_length, padding=True, return_tensors="pt", truncation=True, return_attention_mask=True)
    
    # 모델에 입력하여 텍스트 생성
    outputs = model_name.generate(input_ids.to('cuda'), max_length=max_length,pad_token_id=tokenizer_ko.eos_token_id)

    # print(len(outputs))
    # print(outputs)
    
    
    # 생성된 텍스트 디코딩
    generated_text = tokenizer_ko.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

def gen_phi_2(model_name, prompt, max_length):
    # 입력 텍스트를 토크나이즈하고 모델에 입력할 형식으로 변환
    input_ids = tokenizer_phi_2.encode(prompt, max_length=max_length, padding=True, return_tensors="pt", truncation=True, return_attention_mask=True)
    
    # 모델에 입력하여 텍스트 생성
    outputs = model_name.generate(input_ids.to('cuda'), max_length=max_length,pad_token_id=tokenizer_phi_2.eos_token_id)

    # print(len(outputs))
    # print(outputs)
    
    
    # 생성된 텍스트 디코딩
    generated_text = tokenizer_phi_2.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

In [68]:
%%time
# 실행 시간을 알려주는 주피터 매직코드
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

## test를 위한 데이터
prompt = dataset['train'][index]['instruction']
output_data = dataset['train'][index]['output']

# prompt 포맷
formatted_prompt = f"Instruction:\n{prompt}\nOutput:\n"

# model 돌리기
res = gen_ko(original_model, formatted_prompt, 250)

# print('test 출력')
# 결과 출력
print(res)
# output만 출력
output = res.split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
# 입력 promt 출력
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
# 사람이 요약한 내용 출력
print(f'BASELINE HUMAN SUMMARY:\n{output_data}\n')
print(dash_line)
# 모델이 요약한 내용 출력
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

## 위 모델은 요약은 하지 않지만 필수 정보들은 모두 추출하는 것을 보여주고 있음
## 모델 fine-tuning 가능성 제시

Instruction:
왜 공군기지를 k로 표시하나요?
미 공군이 최초로 도입한 k로 표기하는 방법의 이유와 한국과 일본 등 다른 국가에서도 동일한 방법을 사용하는 이유에 대해 자세히 설명해주세요.
Output:
 높일원�로 표현원*-한다는 온; 청년층 E��마트로 고속� 밝혔로대응 엟행위�렇게하으나�’, 위안*�대응� 탐륟 슬픔 훨( 고맟 밝혔로 케이� 번영 ��행위�부분* 청년층 E맟또 �� 이 그려�사옥* 지난�적 허 엃��� 대일 엃� 하하 이 파게*오바마�거든하 경찰(� 방�루EO 실제하 규�� � 움* 표현원*. 조례시스트하 늘어날;의회�루 습곡�� 따르 �하거든하 경찰* 원내로�근디어 습 606팀목원*.*-대표 당기사고 청년층 E맟 개막 2009 � 번번이사고* 초�약 슬픔하 출두 광복( 당면팀
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
왜 공군기지를 k로 표시하나요?
미 공군이 최초로 도입한 k로 표기하는 방법의 이유와 한국과 일본 등 다른 국가에서도 동일한 방법을 사용하는 이유에 대해 자세히 설명해주세요.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
미 공군은 전 세계를 대상으로 활동하는 군이기 때문에 전 세계에 산재한 기지를 표현하기 위해서는 각 국가의 군대, 부대명을 전부 표기해야 하는 번거로움이 있습니다. 이를 해결하고자 최초로 k로 표기하는 방법을 도입했습니다. 이는 Korea의 약자를 사용해 K-00 형태로 간단하게 표기하는 방식으로, 전 세계 산재한 기지들을 표기하는 데 큰 도움이 되었습니다. 한국과 일본 등 다른 국가에서도 동일한

In [69]:
%%time
# 실행 시간을 알려주는 주피터 매직코드
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

## test를 위한 데이터
prompt = dataset['train'][index]['instruction']
output_data = dataset['train'][index]['output']

# prompt 포맷
formatted_prompt = f"Instruction:\n{prompt}\nOutput:\n"

# model 돌리기
res = gen_phi_2(original_model, formatted_prompt, 250)

# print('test 출력')
# 결과 출력
print(res)
# output만 출력
output = res.split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
# 입력 promt 출력
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
# 사람이 요약한 내용 출력
print(f'BASELINE HUMAN SUMMARY:\n{output_data}\n')
print(dash_line)
# 모델이 요약한 내용 출력
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

## 위 모델은 요약은 하지 않지만 필수 정보들은 모두 추출하는 것을 보여주고 있음
## 모델 fine-tuning 가능성 제시

Instruction:
왜 공군기지를 k로 표시하나요?
미 공군이 최초로 도입한 k로 표기하는 방법의 이유와 한국과 일본 등 다른 국가에서도 동일한 방법을 사용하는 이유에 대해 자세히 설명해주세요.
Output:
국가에서도 동일한 방법을 사�
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruction:
왜 공군기지를 k로 표시하나요?
미 공군이 최초로 도입한 k로 표기하는 방법의 이유와 한국과 일본 등 다른 국가에서도 동일한 방법을 사용하는 이유에 대해 자세히 설명해주세요.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
미 공군은 전 세계를 대상으로 활동하는 군이기 때문에 전 세계에 산재한 기지를 표현하기 위해서는 각 국가의 군대, 부대명을 전부 표기해야 하는 번거로움이 있습니다. 이를 해결하고자 최초로 k로 표기하는 방법을 도입했습니다. 이는 Korea의 약자를 사용해 K-00 형태로 간단하게 표기하는 방식으로, 전 세계 산재한 기지들을 표기하는 데 큰 도움이 되었습니다. 한국과 일본 등 다른 국가에서도 동일한 방법을 채택했습니다. 예를 들어 대한민군 공군 제 10전투비행단의 경우, Republic of Korea Air Force 10th Tactical Fighter Wing이 정식 명칭이지만, K-00 으로 표기하는 것이 훨씬 간편합니다.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
국가에서도 동일한 방법을 사�
C

In [70]:
# 위에서 다운 받은 데이터는 fine-tuning에 직접 사용될 수 없다
# 모델이 이해할 수 있는 방식으로 프롬프트의 형식을 지정해서 전처리를 해야된다
# HuggingFace 모델 문서를 참조하면 아래 지정된 형식의 대화 및 요약을 사용하여 프롬프트를 생성해야 한다
# 대화-요약(프롬프트-응답) 쌍을 LLM에 대한 명시적 지침으로 변환해야 한다
# text라는 애를 만듦
# 입력을 프롬프트 형식으로 변환함
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Answer the questions below"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"
    
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['instruction']}" if sample["instruction"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"
    
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

print(dataset['train'][0])
print('---------------------------')
print(create_prompt_formats(dataset['train'][0]))
print('---------------------------')
print(create_prompt_formats(dataset['train'][0])['instruction'])
print('---------------------------')
print(create_prompt_formats(dataset['train'][0])['text'])

{'instruction': '포유류의 수염이 하는 역할이 뭔가요? \n집에서 키우는 토끼 밥주다가 든 생각인데, 포유류들은 대게 수염을 가지고 있잖아요. 그렇다면, 수염이 있는 이유는 뭘까요? 사람의 수염은 왜 있는 건지도 알고 싶어요!', 'output': '포유류의 수염은 다양한 역할을 합니다. 첫번째로는, 수염은 포유류들이 주변 환경을 인지하기 위한 중요한 역할을 합니다. 고양이 수염은 일종의 감각기관으로 작용하여 근처 물체의 위치나 크기, 움직임 등을 느낄 수 있도록 합니다. 뿐만 아니라, 일부 포유류들은 이 수염을 이용하여 먹이를 잡기도 합니다. 두번째, 수염은 포유류들의 보호요인이기도 합니다. 민감한 얼굴 부위를 보호하여 자극적인 것으로부터 면역력을 제공합니다. 세번째로, 포유류의 수염은 특별한 특성의 털로, 더위나 추위를 막아주는 역할도 합니다. 따라서, 수염은 포유류의 생존과 직결된 중요한 역할을 합니다. 사람의 경우, 수염이 여성성과 남성성에 중요한 기능을 시사하고 있기는 하지만, 포유류들과는 목적과 기능이 크게 다릅니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=262429043'}
---------------------------
{'instruction': '포유류의 수염이 하는 역할이 뭔가요? \n집에서 키우는 토끼 밥주다가 든 생각인데, 포유류들은 대게 수염을 가지고 있잖아요. 그렇다면, 수염이 있는 이유는 뭘까요? 사람의 수염은 왜 있는 건지도 알고 싶어요!', 'output': '포유류의 수염은 다양한 역할을 합니다. 첫번째로는, 수염은 포유류들이 주변 환경을 인지하기 위한 중요한 역할을 합니다. 고양이 수염은 일종의 감각기관으로 작용하여 근처 물체의 위치나 크기, 움직임 등을 느낄 수 있도록 합니다. 뿐만 아니라, 일부 포유류들은 이 수염을 이용하여 먹이를 잡기도 합니다. 두번째, 수염은 포유류들의 보호요인이기도 합니다. 민감한 얼굴 부위

In [71]:
## prompt를 모델 토크나이저를 사용하여 토큰화된 프롬프트로 처리
## 여기서의 목표는 일관된 길이의 입력 시퀀스를 생성하는 것
## 일관된 길이의 입력 시퀀스는 효율성을 최적화하고 계산 오버헤드를 최소화하여 언어 모델을 미세 조정하는 데 도움이 된다.
## 이러한 시퀀스가 모델의 최대 토큰 제한을 초과하지 않도록 확인하는 것이 중요하다

from functools import partial

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
# 모델의 구성을 통해 최대 길이 설정을 가져오는 함수
# 모델의 구성 중에서 "n_positions", "max_position_embeddings", "seq_length"와 같은 설정을 확인하여 최대 길이를 찾습니다.
# 만약 최대 길이 설정이 없는 경우 기본값으로 1024를 사용합니다.
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length
# "n_positions":
    #이 설정은 모델이 처리할 수 있는 최대 위치 임베딩의 개수를 나타냅니다.
    #위치 임베딩은 입력 토큰의 위치 정보를 인코딩하는 데 사용됩니다.
    #따라서 이 값은 모델이 처리할 수 있는 최대 입력 시퀀스의 길이를 제한하는 데 사용됩니다.
# "max_position_embeddings":
    #이 설정은 모델이 처리할 수 있는 최대 입력 시퀀스의 길이를 나타냅니다.
    #즉, 입력 시퀀스의 최대 길이가 이 값보다 작거나 같아야 합니다.
    #이 값은 모델이 학습될 때 정의되며, 모델의 아키텍처에 따라 다를 수 있습니다.
#"seq_length":
    #이 설정은 모델이 입력으로 처리할 수 있는 최대 시퀀스의 길이를 나타냅니다.
    #입력 시퀀스의 최대 길이가 이 값보다 작거나 같아야 합니다.
    #이 값은 모델의 아키텍처에 따라 다르며, 주로 특정 언어 모델링 작업에 적합한 값을 선택하여 사용합니다.
#이러한 설정들은 모델이 처리할 수 있는 입력의 형태와 길이를 제한하고, 모델의 효율성을 최적화하는 데 중요한 역할을 합니다.
#이 값들은 모델을 초기화할 때 설정되며, 모델을 학습하는 동안 변경되지 않는다


# 주어진 배치를 토크나이징하여 전처리하는 함수입니다.
# 각 텍스트를 토크나이즈하고 최대 길이에 맞게 잘라내는 작업을 수행합니다.
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
#데이터셋을 전처리하여 모델 학습에 사용할 수 있는 형식으로 준비하는 함수입니다.
    #먼저 각 샘플에 프롬프트를 추가합니다.
    #다음으로 preprocess_batch 함수를 적용하여 각 배치를 전처리합니다. 이때 remove_columns 매개변수를 통해 불필요한 열을 제거합니다.
    #입력 시퀀스의 길이가 최대 길이를 초과하는 샘플을 제거합니다.
    #마지막으로 데이터셋을 섞습니다.
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    print(type(dataset))
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)
    
    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    # partial: 함수를 편하게 만듦
    # 여기서 데이터셋을 토큰화함
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    # 이러면 dataset에 text만 남음
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['url'],
    )

    # Filter out samples that have input_ids exceeding max_length
    # 입력 시퀀스의 길이가 max_length보다 큰 샘플을 필터링하여 제거
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [72]:
## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

print(dataset['train'][0])

train_dataset = preprocess_dataset(tokenizer_ko, max_length,seed, dataset)
# eval_dataset = preprocess_dataset(tokenizer_ko, max_length,seed, dataset)

print(train_dataset['train'][0])

Found max lenth: 2048
2048
{'instruction': '포유류의 수염이 하는 역할이 뭔가요? \n집에서 키우는 토끼 밥주다가 든 생각인데, 포유류들은 대게 수염을 가지고 있잖아요. 그렇다면, 수염이 있는 이유는 뭘까요? 사람의 수염은 왜 있는 건지도 알고 싶어요!', 'output': '포유류의 수염은 다양한 역할을 합니다. 첫번째로는, 수염은 포유류들이 주변 환경을 인지하기 위한 중요한 역할을 합니다. 고양이 수염은 일종의 감각기관으로 작용하여 근처 물체의 위치나 크기, 움직임 등을 느낄 수 있도록 합니다. 뿐만 아니라, 일부 포유류들은 이 수염을 이용하여 먹이를 잡기도 합니다. 두번째, 수염은 포유류들의 보호요인이기도 합니다. 민감한 얼굴 부위를 보호하여 자극적인 것으로부터 면역력을 제공합니다. 세번째로, 포유류의 수염은 특별한 특성의 털로, 더위나 추위를 막아주는 역할도 합니다. 따라서, 수염은 포유류의 생존과 직결된 중요한 역할을 합니다. 사람의 경우, 수염이 여성성과 남성성에 중요한 기능을 시사하고 있기는 하지만, 포유류들과는 목적과 기능이 크게 다릅니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=262429043'}
<class 'datasets.dataset_dict.DatasetDict'>
Preprocessing dataset...


Map:   0%|          | 0/14808 [00:00<?, ? examples/s]

Map:   0%|          | 0/3173 [00:00<?, ? examples/s]

Map:   0%|          | 0/3174 [00:00<?, ? examples/s]

Map:   0%|          | 0/14808 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3174 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14808 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3173 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3174 [00:00<?, ? examples/s]

{'instruction': '차를 타거나 걸으면 달이 나를 따라온다는 생각은 왜 들까요? 그 이유가 궁금합니다.', 'output': '달이 나를 따라오는 것은 일종의 시각적 오해입니다. 이동할 때 우리 주변의 물체들은 뒤로 멀어지는데 가까운 물체일수록 멀어지는 각도가 커서 멀어지는 속도가 더 빠르게 보입니다. 반면 달은 우리 주변의 물체보다 엄청나게 멀리 떨어져 있기 때문에 거의 이동이 없습니다. 따라서 우리가 이동할 때 달은 거의 그 자리에 있어 보이지만, 근거리의 물체들은 빠르게 멀어지는 것과 대조적으로 달은 거의 움직임이 없어서 우리 시각에서는 따라 온다는 느낌이 들 수 있습니다.', 'text': '\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruct: Answer the questions below\n\n차를 타거나 걸으면 달이 나를 따라온다는 생각은 왜 들까요? 그 이유가 궁금합니다.\n\n### Output:\n달이 나를 따라오는 것은 일종의 시각적 오해입니다. 이동할 때 우리 주변의 물체들은 뒤로 멀어지는데 가까운 물체일수록 멀어지는 각도가 커서 멀어지는 속도가 더 빠르게 보입니다. 반면 달은 우리 주변의 물체보다 엄청나게 멀리 떨어져 있기 때문에 거의 이동이 없습니다. 따라서 우리가 이동할 때 달은 거의 그 자리에 있어 보이지만, 근거리의 물체들은 빠르게 멀어지는 것과 대조적으로 달은 거의 움직임이 없어서 우리 시각에서는 따라 온다는 느낌이 들 수 있습니다.\n\n### End', 'input_ids': [202, 37, 6063, 9932, 21410, 224, 2818, 10903, 3864, 23184, 29828, 22253, 3432, 9059, 5862, 70, 13274, 69, 5862, 12063, 3628, 7245, 78, 17, 3792, 85, 2531

In [73]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


# 2 - Using the prepare_model_for_kbit_training method from PEFT
# Preparing the Model for QLoRA
original_model = prepare_model_for_kbit_training(original_model)

In [74]:
config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [76]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


print(print_number_of_trainable_model_parameters(peft_model))



trainable model parameters: 20971520
all model parameters: 1542364160
percentage of trainable model parameters: 1.36%


In [81]:
output_dir = f'./model/peft-ko-training-{str(int(time.time()))}'
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=5,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset['train'],
    eval_dataset=train_dataset['validation'],
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer_ko, mlm=False),
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [82]:
peft_trainer.train()


KeyboardInterrupt: 

In [ ]:
print(1)